# People Analytics - Análise Exploratória

Este notebook permite analisar os dados de avaliações, diagnosticar problemas e visualizar resultados.

In [ ]:
# Importações necessárias
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
from IPython.display import display, HTML

# Configuração para exibição de dados
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

# Configuração de estilo para visualizações
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

## Definição de funções úteis

In [ ]:
def load_json_file(file_path):
    """Carrega um arquivo JSON com tratamento de erros"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f), None
    except json.JSONDecodeError as e:
        return None, f"Erro ao decodificar JSON: {str(e)}"
    except Exception as e:
        return None, f"Erro ao ler arquivo: {str(e)}"

def scan_directory(directory_path, pattern="*/*/resultado.json"):
    """Escaneia diretório procurando arquivos que seguem um padrão"""
    path = Path(directory_path)
    files = list(path.glob(pattern))
    return files

def diagnose_json_file(file_path):
    """Diagnostica problemas em um arquivo JSON de avaliação"""
    data, error = load_json_file(file_path)
    if error:
        return {"file": str(file_path), "status": "error", "message": error}
    
    issues = []
    
    # Verificar estrutura básica
    if 'data' not in data:
        issues.append("Campo 'data' ausente")
    elif 'direcionadores' not in data['data']:
        issues.append("Campo 'direcionadores' ausente")
    elif not data['data']['direcionadores']:
        issues.append("Lista de direcionadores vazia")
    else:
        # Verificar cada direcionador e comportamento
        for i, direcionador in enumerate(data['data']['direcionadores']):
            if 'comportamentos' not in direcionador:
                issues.append(f"Direcionador {i+1} não tem campo 'comportamentos'")
                continue
                
            for j, comportamento in enumerate(direcionador.get('comportamentos', [])):
                if 'avaliacoes_grupo' not in comportamento:
                    issues.append(f"Comportamento {j+1} em Direcionador {i+1} não tem campo 'avaliacoes_grupo'")
                    continue
                    
                for k, avaliacao in enumerate(comportamento.get('avaliacoes_grupo', [])):
                    if 'frequencia_colaborador' not in avaliacao:
                        issues.append(f"Avaliação {k+1} em Comportamento {j+1} não tem campo 'frequencia_colaborador'")
                    elif not isinstance(avaliacao.get('frequencia_colaborador'), list):
                        issues.append(f"Campo 'frequencia_colaborador' não é uma lista em Avaliação {k+1}")
                        
                    if 'frequencia_grupo' not in avaliacao:
                        issues.append(f"Avaliação {k+1} em Comportamento {j+1} não tem campo 'frequencia_grupo'")
                    elif not isinstance(avaliacao.get('frequencia_grupo'), list):
                        issues.append(f"Campo 'frequencia_grupo' não é uma lista em Avaliação {k+1}")
    
    return {
        "file": str(file_path),
        "status": "ok" if not issues else "issues",
        "issues": issues,
        "data": data
    }

## Diretório dos Dados

Configure o caminho para o diretório de dados:

In [ ]:
# Configurar caminho para diretório de dados
DATA_DIR = "teste"  # Altere para o caminho dos seus dados reais

# Encontrar arquivos no padrão <pessoa>/<ano>/resultado.json
files = scan_directory(DATA_DIR, "*/*/resultado.json")
print(f"Encontrados {len(files)} arquivos de resultados:")
for f in files:
    print(f"  - {f}")